In [29]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import FlightSimTVC
import FlightSimAero
import FlightSimAtt
%matplotlib widget

In [4]:
# Constants
drop_height = 20                    # meters
fin_area = 0.0065366                # area of one fin [m^2]
lever_f = np.array([0,0,0.26])      # vector from cg of fin to cp (meters) 11/10/22
lever_c = np.array([0,0,0.08671])   # vector from cg of body/legs/cap to cp (meters) KEEP SAME

In [32]:
# Define main simulation function
def simulate(t_ignite,Kp,Kd,drop_height,fin_area,lever_c,lever_f,wind,omega_init):

    dt = 0.01                       # time step
    t_sim = np.arange(0,4,dt)       # time range for simulation
    loops = len(t_sim)              # number of steps

    # arrays for collecting data
    acc = np.zeros((loops,3))                   # acceleration vector of rocket
    v = np.zeros((loops,3))                     # velocity vector of rocket      
    p = np.zeros((loops,3))                     # position vector of rocket
    q = np.zeros((loops,4))                     # quaternion of rocket attitude
    omega = np.zeros((loops,3))                 # angular velocity of rocket
    aero_torque = np.zeros((loops,3))
    aero_force = np.zeros((loops,3))
    thrust_force = np.zeros((loops,3))          # body frame
    thrust_torque = np.zeros((loops,3))         # body frame
    torque = np.zeros((loops,3))
    z_i = np.zeros((loops,3))                   # unit vector of z axis of rocket in fixed frame
    z_angle = np.zeros(loops)                   # rocket angle from vertical

    # Initial Conditions
    m = 0.95                                                # rocket initial mass (kg)
    g = np.array([ 0, 0,-9.81])                             # gravitational acceleration (m/s^2), Z axis is up
    acc[0,:] = g                                            # acceleration (m/s^2)
    v[0,:] = np.array([0.00001,0.0000001,0.000001])         # velocity (m/s)
    p[0,:] = np.array([0,0,drop_height])                    # position (m)
    omega[0,:] = omega_init                                 # angular rate (rad/sec)
    torque[0,:] = np.zeros(3)                               # (N-m)
    

    z_i[0,:] = np.zeros(3)

    r = R.from_euler('ZYX',np.array([0, 0, 0]))
    q0 = r.as_quat().T
    q[0,:] = q0    

    # Rocket model parameters
    feet_area_v = (np.pi*0.025**2) * 4
    legs_area_v = (0.03*0.14) *4
    tube_circle = np.pi*0.04**2
    base_area = feet_area_v+legs_area_v+tube_circle         # vertically projected area of rocket
    legs_area_s = (0.01*0.160+0.05*0.06) *2
    tube_diameter = 0.07874
    length = 0.3302
    tube_diameter = 0.07874
    tube_side_area = length * tube_diameter
    side_area = tube_side_area + 2*fin_area  + legs_area_s  # horizontally projected area of rocket


    cd_base = 1.15                                          # drag coeff of rocket in pure vertical descent (experimentally determined)
    cd_side = 1.28                                          # drag coeff of rocket in pure horizantal descent with 2 fins perpendicular to flow (approximated)

    # Engine parameters for mass calculation
    net_impulse = 13.8                      # net impulse from ThrustCurve.com (Ns)
    m_propellant = 0.009                    # propellant of mass from ThrustCurve.com (kg)
    Isp = net_impulse/(m_propellant*9.81)   # specific impulse (s)

    # initialize servo servo angles
    phi_save = np.zeros(loops)
    theta_save = np.zeros(loops)

    # create attitude_simulator object 
    att_sim_obj = FlightSimAtt.attitude_simulator( q[0,:], omega[0,:], dt )

    # create aero_simulator object
    aero_sim_obj = FlightSimAero.aero_simulator( lever_f, lever_c, fin_area, side_area)

    # create tvc_simulator object
    tvc_sim_obj = FlightSimTVC.tvc_simulator(dt,Kp,Kd)
    burn_time = tvc_sim_obj.t_thrust[-1]


    # loop through time step and simulate dynamics
    i = 0     # index i tracks simulation time step
    j = 0     # index j tracks time step within thrust curve
    while i < loops-1:

        #------- run attitude simulator --------%
        # this updates the rocket orientation for the next time step
        att_sim_obj.att_RK4( torque[i,:] )
        omega[i+1,:] = att_sim_obj.w
        q[i+1,:] = att_sim_obj.q
        rot = R.as_matrix(R.from_quat([q[i+1,1],q[i+1,2],q[i+1,3],q[i+1,0]]))
    
        #-------- run aero simulator --------%
        aero_torque[i,:], aero_force[i,:], z_i[i,:], z_angle[i] = aero_sim_obj.aero_3d( rot, v[i,:], omega[i,:], q[i,:],cd_base, cd_side, base_area, side_area, wind )

        #-------- free fall condition --------%
        # note: used both before and after burn
        if t_sim[i] < t_ignite or t_sim[i] >= t_ignite + burn_time:
            # no thrust so...
            thrust_force[i,:] = np.zeros(3)
            thrust_torque[i,:] = np.zeros(3)
            phi_save[i] = 0
            theta_save[i] = 0     # Define as zero after engine stops thrusting

            # translation motion
            # update rocket position for next time step based on current position and applicable forces
            acc[i+1,:] = g + aero_force[i,:] / m
            p[i+1,:] = p[i,:] + v[i,:] * dt + 0.5 * acc[i+1,:] * dt**2
            v[i+1,:] = v[i,:] + acc[i+1,:] * dt

        #-------- power descent --------%
        else:
            # print(f"time: {t_sim[i]})")
            # Control in general is calculated about every 0.04 seconds
            if j == 0 or j%4 == 0:
                thrust_torque[i,:], thrust_force[i,:], phi_save[i], theta_save[i] = tvc_sim_obj.tvc_control( q[i-3,:], omega[i- 3,:], j, phi_save[i-1],theta_save[i-1] )
        
            # between these larger control steps, the control output is maintained at the most recent values returned by tvc_control
            else:                   
                thrust_force[i,:] = thrust_force[i-1,:]
                thrust_torque[i,:] = thrust_torque[i-1,:]
                theta_save[i] = theta_save[i-1]
                phi_save[i] = phi_save[i-1]

            # translation motion
            # update rocket position for next time step based on current position and applicable forces
            acc[i+1,:] = g + rot @ thrust_force[i,:]/m + aero_force[i,:] / m    # rotate thrust force into inertial frame
            p[i+1,:] = p[i,:] + v[i,:] * dt + 0.5 * acc[i,:] * dt**2
            v[i+1,:] = v[i,:] + acc[i+1,:] * dt
            # update mass
            mdot = -tvc_sim_obj.thrust_curve[j]/(9.81*Isp)
            m = m + mdot * dt
            # update thrust time step
            j += 1
    

        # update total torque acting on rocket 
        torque[i+1,:] = thrust_torque[i,:] + aero_torque[i,:]

        # update simulation time step
        i += 1

        #-------- landed --------%
        if  p[i,2] <= 0.:
            #trim remaining zeros...
            p = p[:i,:]
            v = v[:i,:]
            acc = acc[:i,:]
            z_i = z_i[:i,:]
            z_angle = z_angle[:i]
            q = q[:i,:]
            phi_save = phi_save[:i]
            theta_save = theta_save[:i]
            #...and exit the while loop
            break 

    return (p,v,z_angle,phi_save,theta_save,t_sim)

In [69]:
def cost(params, max_wind=1.4, max_angvel=3, num_sims=20):

    (t_ignite, Kp, Kd) = params

    net_v = 0
    net_ang = 0

    rand3s = np.random.multivariate_normal([0, 0, 0], np.diag([0.5, 0.5, 0.1]), num_sims+1)
    for i in range(num_sims):
        wind = rand3s[i]
        omega_init = rand3s[i+1]*0.1
        print(wind, omega_init)

        
        (p,v,z_angle,phi_save,theta_save,t_sim) = simulate(t_ignite, Kp, Kd, drop_height, fin_area, lever_c, lever_f,wind,omega_init)
        # ang_int = np.trapz([z_angle])
        net_v += -v[-1,2]
        net_ang += z_angle[-1]

        # print(sim_cost)
    avg_v = net_v/num_sims
    avg_ang = net_ang/num_sims * 1000
    print(f"avg v: {avg_v}")
    print(f"avg ang_int: {avg_ang}")
    net_cost = avg_v + avg_ang
    return -net_cost
    

params = np.array([ 1.20935806e+00,  1.59841540e-02, -5.62940158e-04])
cost(params)
        



[-0.1941511   0.64678625  0.15601048] [ 0.03035446 -0.09025096 -0.02844222]
[ 0.3035446  -0.90250956 -0.28442223] [-0.02590776  0.16581879  0.03634627]
[-0.25907756  1.65818792  0.36346271] [-0.01320738 -0.02465741 -0.02558897]
[-0.13207377 -0.24657406 -0.25588965] [-0.03021341  0.00307577  0.02283685]
[-0.30213408  0.0307577   0.22836849] [-0.03012886 -0.03884896  0.02975978]
[-0.30128857 -0.38848963  0.29759778] [-0.04945417 -0.05470671  0.01499764]
[-0.49454171 -0.54706715  0.1499764 ] [ 0.08514109 -0.04180359 -0.00253349]
[ 0.85141093 -0.41803585 -0.02533492] [-0.0576573  -0.00562974  0.05466892]
[-0.57657297 -0.05629738  0.54668919] [ 0.06548477  0.14310751 -0.03607064]
[ 0.65484774  1.43107509 -0.3607064 ] [ 0.00407776 -0.09526011 -0.00052177]
[ 0.04077756 -0.95260113 -0.00521768] [-0.03097852 -0.0231933  -0.00018942]
[-0.30978519 -0.23193305 -0.00189419] [ 0.0534336  -0.01270111  0.03459351]
[ 0.53433595 -0.1270111   0.34593506] [-0.01215893  0.01363421 -0.0049083 ]
[-0.12158927

-77.79453208682234

In [48]:
x0 = np.array([1.2, 0.05, 0])
bounds = np.array([[1.1,1.3], [0.0,0.1], [0.0, 0.1]])
res = scipy.optimize.minimize(cost, x0, bounds=bounds, options={'disp':True}, tol=1e-2)
print(res['x'])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f=  1.44714D+01    |proj g|=  1.00000D-01

At iterate    1    f=  1.32625D+01    |proj g|=  9.99974D-02
[1.20000264 0.05000131 0.        ]
At iterate    2    f=  1.39103D+01    |proj g|=  1.00003D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      2     19      4     0     1   1.000D-01   1.391D+01
  F =   13.910318151603525     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



In [49]:
res.

      fun: 13.910318151603525
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.04200007e+08, 9.26114297e+07, 6.73990245e+07])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 76
      nit: 2
     njev: 19
   status: 0
  success: True
        x: array([1.20000264, 0.05000131, 0.        ])